In [1]:
import pandas as pd

In [11]:
df1 = pd.read_parquet('../data/raw/yellow_tripdata_2024-01.parquet')
df2 = pd.read_parquet('../data/raw/yellow_tripdata_2024-02.parquet')
df3 = pd.read_parquet('../data/raw/yellow_tripdata_2024-03.parquet')
df4 = pd.read_parquet('../data/raw/yellow_tripdata_2024-04.parquet')
df5 = pd.read_parquet('../data/raw/yellow_tripdata_2024-05.parquet')
df6 = pd.read_parquet('../data/raw/yellow_tripdata_2024-06.parquet')

In [12]:
df = pd.concat([df1, df2, df3, df4, df5, df6])

In [13]:
df.shape

(20332093, 19)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20332093 entries, 0 to 3539192
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee            flo

In [16]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


In [17]:
missing_values = df.isnull().sum()
missing_values

VendorID                       0
tpep_pickup_datetime           0
tpep_dropoff_datetime          0
passenger_count          1975985
trip_distance                  0
RatecodeID               1975985
store_and_fwd_flag       1975985
PULocationID                   0
DOLocationID                   0
payment_type                   0
fare_amount                    0
extra                          0
mta_tax                        0
tip_amount                     0
tolls_amount                   0
improvement_surcharge          0
total_amount                   0
congestion_surcharge     1975985
Airport_fee              1975985
dtype: int64

In [18]:
unique_values = df['passenger_count'].unique()
unique_values

array([ 1.,  2.,  0.,  4.,  3.,  5.,  6.,  8.,  7.,  9., nan])